### Experiment with MongoEngine

The ODM we use to access MongoDB

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document
from dexter.config import Tag

from datetime import date

Open the database:

In [2]:
DB.open('pytest')

Make an account:

In [3]:
acct = Account(name='equity', category='equity')

Save it:

In [4]:
acct.save()

<Account: <Acct equity equity>>

If we open that DB with `mongosh` we should see the account.

```
$ mongosh

test> use foo
switched to db foo

foo> db.account.find()
[
  {
    _id: ObjectId('67c61fa19d0161a19b80469e'),
    name: 'equity',
    group: 'equity'
  }
]
```

It worked!  🎉

### Contents of a Collection

In [5]:
Account.objects

[<Account: <Acct equity equity>>, <Account: <Acct yoyodyne income>>, <Account: <Acct bank:checking asset>>, <Account: <Acct amex:blue liability>>, <Account: <Acct chase:visa liability>>, <Account: <Acct groceries expense>>, <Account: <Acct household expense>>, <Account: <Acct mortgage expense>>, <Account: <Acct car expense>>, <Account: <Acct travel expense>>, <Account: <Acct equity equity>>, <Account: <Acct equity equity>>]

In [6]:
Account.objects[0]

<Account: <Acct equity equity>>

In [7]:
acct = Account.objects[0]

In [8]:
acct.name

'equity'

In [9]:
acct['name']

'equity'

### Low Level API

We can also connect to the DB directly to use the `pymongo` library, _e.g._ to get collection names.

After calling `DB.open` we can get a reference to the client and the current database using static vars of the module:

In [10]:
DB.client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None))

In [11]:
DB.client.list_database_names()

['admin', 'config', 'dev', 'dexter', 'local', 'pytest']

In [12]:
DB.database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest')

In [13]:
db = DB.database

In [14]:
db.account

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [15]:
db.account.find_one()

{'_id': ObjectId('67f16179bcd0bf7da28fa44f'),
 'name': 'equity',
 'category': 'equity',
 'comment': 'type: equity'}

In [16]:
for c in db.list_collections():
    print(c)

{'name': 'entry', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'<j\xa3Z\x13\xf3F\x8d\xb7\x9fJ\xb7\xde\x8a\x07\xe3', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'transaction', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b"_\x1e\xb9c0\xa9A2\xa2'\xb9\xd5\x85D \x14", 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'reg_exp', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xa2\xf9\xee\xa4\xed*Ja\x97\rl\xceDA\xb7\xaf', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'account', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xaa\xfe\xa4l\x96XAo\x93\xd0\x82\x00{\xba\x7fe', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}


In [17]:
for name in db.list_collection_names():
    print(name)

entry
transaction
reg_exp
account


In [18]:
db['account']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [19]:
db['account'].find_one()

{'_id': ObjectId('67f16179bcd0bf7da28fa44f'),
 'name': 'equity',
 'category': 'equity',
 'comment': 'type: equity'}

In [20]:
for obj in db['account'].find():
    print(obj)

{'_id': ObjectId('67f16179bcd0bf7da28fa44f'), 'name': 'equity', 'category': 'equity', 'comment': 'type: equity'}
{'_id': ObjectId('67f16179bcd0bf7da28fa450'), 'name': 'yoyodyne', 'category': 'income', 'comment': 'type: income'}
{'_id': ObjectId('67f16179bcd0bf7da28fa451'), 'name': 'bank:checking', 'category': 'asset', 'comment': 'type: asset'}
{'_id': ObjectId('67f16179bcd0bf7da28fa452'), 'name': 'amex:blue', 'category': 'liability', 'comment': 'type: liability'}
{'_id': ObjectId('67f16179bcd0bf7da28fa453'), 'name': 'chase:visa', 'category': 'liability', 'comment': 'type: liability'}
{'_id': ObjectId('67f16179bcd0bf7da28fa454'), 'name': 'groceries', 'category': 'expense', 'comment': 'type: expense'}
{'_id': ObjectId('67f16179bcd0bf7da28fa455'), 'name': 'household', 'category': 'expense', 'comment': 'type: expense'}
{'_id': ObjectId('67f16179bcd0bf7da28fa456'), 'name': 'mortgage', 'category': 'expense', 'comment': 'type: expense'}
{'_id': ObjectId('67f16179bcd0bf7da28fa457'), 'name': 'c

### From Low Level to High Level

Question:  given a collection name ("account") can we find the corresponding MongoEngine class (Account)?

In [21]:
Document

mongoengine.document.Document

In [22]:
Document.__subclasses__()

[mongoengine.document.DynamicDocument,
 dexter.DB.Account,
 dexter.DB.Entry,
 dexter.DB.Transaction,
 dexter.DB.RegExp]

In [23]:
[cls for cls in Document.__subclasses__() if hasattr(cls, 'objects')]

[dexter.DB.Account, dexter.DB.Entry, dexter.DB.Transaction, dexter.DB.RegExp]

In [24]:
Account._meta

{'abstract': False,
 'max_documents': None,
 'max_size': None,
 'ordering': [],
 'indexes': [],
 'id_field': 'id',
 'index_background': False,
 'index_opts': None,
 'delete_rules': None,
 'allow_inheritance': None,
 'collection': 'account',
 'index_specs': []}

In [25]:
for cls in Document.__subclasses__():
    if not hasattr(cls, 'objects'):
        continue
    print(cls._meta['collection'], cls)

account <class 'dexter.DB.Account'>
entry <class 'dexter.DB.Entry'>
transaction <class 'dexter.DB.Transaction'>
reg_exp <class 'dexter.DB.RegExp'>


### The Big Picture

Use the high level API when working with data.  MongoEngine converts the documents into objects (which is something we'd be doing ourselves if we didn't use it).

Use the low level API for collective operations: exporting, importing, ...

**NOTE**  It's possible to get a document using the low level API, as shown above, but it will be a `dict`, not a model instance.

### Transactions

In [26]:
t = Transaction(description='hi', comment='aloha')

In [27]:
t.description

'hi'

Nice -- the list fields are initially empty.

In [28]:
t.tags

[]

In [29]:
t.entries

[]

### Entries

In [30]:
e = Entry(uid='xxx', column='credit', date='2025-03-05', amount=1000, account='unknown')

In [31]:
type(e)

dexter.DB.Entry

In [32]:
e

<Entry: <En 2025-03-05 unknown -$1000.0>>

In [33]:
e.column

<Column.cr: 'credit'>

In [34]:
e.column.opposite()

<Column.dr: 'debit'>

In [35]:
e.column.opposite().opposite()

<Column.cr: 'credit'>

In [36]:
e.amount

1000.0

In [37]:
e.hash

'6fce51cdae9a1803b7c8d26e12244edc'

In [38]:
len(e.hash)

32

In [39]:
{e.uid for e in Entry.objects}

{'01a6c8b9c5ea155bcf6a294036ee0fef',
 '141acd760303ee498c538ae564cf7bd3',
 '1732e5ef3f64fc8fc28429a867b994f2',
 '1e39a44e4a688705805c48993d8a1c86',
 '20c32394280e4bf00f0faf05aab51d3d',
 '2837965e0b02ce267303b66e66ac9c2c',
 '34bee8ca5e3faf0497a19cf7b1e9e65c',
 '35495d5b5ddb481fa571b055d8aa0c1c',
 '379f5e4e8736f038e5f447cb1ad698dd',
 '39ba02cf76f55e736d48eecaa9e8fd2a',
 '3f734f34bd78107418621a2e01ba8540',
 '48529d349536ef30fdb673e9ccbe2b16',
 '51935235b5aae2032057b14d142d0b22',
 '528bebc00189689329b3e5d54ec73989',
 '55b550d1058f42f0ababbce1e096c420',
 '6f2d599c07e27edc03feeb8bf2acdc80',
 '6ff8eec43005d6834a414d4f307655a9',
 '731e7aa206d4703144631e63852e3ad8',
 '8291d7bf3f1960e028944879cf5f8690',
 '86b576cc0caca598e7947359344d4f6d',
 '87c83a9615a2780e3680c684faa0f073',
 '950f17ee230abc8e769a4b8189c100b9',
 '9a274abe41d85356d391269f0795967b',
 '9c50e74c3edb5c897ad523a4aaa278e0',
 'a1e1ac32faad7a5ac3b74ad2e481ae53',
 'a726755edba7d26587e9b126fb8f0b83',
 'a8841c89a2931321c4e271de5ebb8064',
 

In [40]:
s = set()
for e in Entry.objects:
    if e in s:
        print(e.date, e.amount, e.description)
    s.add(e.uid)

In [41]:
len(s)

38

In [42]:
len(Entry.objects)

38

In [43]:
lst = sorted([e.uid for e in Entry.objects])

In [44]:
len(lst)

38

In [45]:
dup = []
for i in range(len(lst)-1):
    if lst[i] == lst[i+1]:
        dup.append(lst[i])

In [46]:
dup

[]

In [47]:
for e in Entry.objects:
    if e.uid in dup:
        print(e.date, e.amount, e.account, e.description)

### Tags

In [48]:
e = Entry.objects[0]

In [49]:
e.description

'Jan pay'

In [50]:
e.tags

[]

In [51]:
e.tags.append(Tag.U)

In [52]:
e.save()

<Entry: <En 2024-01-02 bank:checking +$5000.0>>

In [53]:
e.tags

[<Tag.U: 'unpaired'>]

In [54]:
e.note = "hello"

In [55]:
e.save()

<Entry: <En 2024-01-02 bank:checking +$5000.0>>

### References

The big test -- can we add that Entry to the transaction?

In [56]:
t.entries.append(e)

In [57]:
t.entries

[<Entry: <En 2024-01-02 bank:checking +$5000.0>>]

Yes!  🎉

### Misc Commands

In [58]:
db.stats

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')

In [59]:
db.stats.find_one

<bound method Collection.find_one of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')>

In [60]:
db.list_collection_names()

['entry', 'transaction', 'reg_exp', 'account']

In [61]:
db.command('count','account')

{'n': 12, 'ok': 1.0}

In [62]:
db.command('hello')

{'isWritablePrimary': True,
 'topologyVersion': {'processId': ObjectId('67f140001cbe7d77833a339b'),
  'counter': 0},
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2025, 4, 6, 17, 16, 51, 168000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 292,
 'minWireVersion': 0,
 'maxWireVersion': 25,
 'readOnly': False,
 'ok': 1.0}

In [63]:
db.command('hostInfo')

{'system': {'currentTime': datetime.datetime(2025, 4, 6, 17, 16, 51, 172000),
  'hostname': 'cthulhu.local',
  'cpuAddrSize': 64,
  'memSizeMB': 65536,
  'memLimitMB': 65536,
  'numCores': 12,
  'numCoresAvailableToProcess': 12,
  'numPhysicalCores': 12,
  'numCpuSockets': 1,
  'cpuArch': 'arm64',
  'numaEnabled': False,
  'numNumaNodes': 1},
 'os': {'type': 'Darwin', 'name': 'Mac OS X', 'version': '24.4.0'},
 'extra': {'versionString': 'Darwin Kernel Version 24.4.0: Wed Mar 19 21:17:25 PDT 2025; root:xnu-11417.101.15~1/RELEASE_ARM64_T6020',
  'alwaysFullSync': 0,
  'nfsAsync': 0,
  'model': 'Mac14,13',
  'cpuString': 'Apple M2 Max',
  'pageSize': 16384,
  'scheduler': 'edge'},
 'ok': 1.0}

In [64]:
db.command('ping')

{'ok': 1.0}

### Fetch Transactions

Specify constraints on transactions

In [65]:
Transaction.objects

[<Transaction: <Tr 2024-01-02 yoyodyne -> bank:checking $5000.0 paycheck>>, <Transaction: <Tr 2024-01-02 groceries/mortgage/car/travel -> yoyodyne $5000.0 fill buckets>>, <Transaction: <Tr 2024-02-02 yoyodyne -> bank:checking $5000.0 paycheck>>, <Transaction: <Tr 2024-02-02 groceries/mortgage/car/travel -> yoyodyne $5000.0 fill buckets>>, <Transaction: <Tr 2024-01-05 bank:checking -> car $500.0 car payment>>, <Transaction: <Tr 2024-02-05 bank:checking -> car $500.0 car payment>>, <Transaction: <Tr 2024-01-10 chase:visa -> car $50.0 Shell Oil>>, <Transaction: <Tr 2024-02-26 chase:visa -> car $60.0 Shell Oil>>, <Transaction: <Tr 2024-01-04 bank:checking -> mortgage $1800.0 Rocket Mortgage>>, <Transaction: <Tr 2024-02-04 bank:checking -> mortgage $1800.0 Rocket Mortgage>>, <Transaction: <Tr 2024-01-07 bank:checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-01-21 chase:visa -> groceries $175.0 Safeway>>, <Transaction: <Tr 2024-02-07 bank:checking -> groceries $75.0 Safeway>>, <T

In [66]:
Transaction.objects(description='Safeway')

[<Transaction: <Tr 2024-01-07 bank:checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-01-21 chase:visa -> groceries $175.0 Safeway>>, <Transaction: <Tr 2024-02-07 bank:checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-02-21 chase:visa -> groceries $75.0 Safeway>>]

In [67]:
for t in Transaction.objects(description='Safeway'):
    for e in t.entries:
        print(e.date, e.account, e.amount, e.column)

2024-01-07 groceries 75.0 debit
2024-01-07 bank:checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 chase:visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 bank:checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 chase:visa 75.0 credit


In [68]:
for t in Transaction.objects(description='Safeway'):
    print(t.accounts)

{'groceries', 'bank:checking'}
{'groceries', 'chase:visa'}
{'groceries', 'bank:checking'}
{'groceries', 'chase:visa'}


In [69]:
for t in Transaction.objects(description='Safeway'):
    print(t.pamount)

75.0
175.0
75.0
75.0


In [70]:
for t in Transaction.objects(description='Safeway'):
    print(t.pdate, type(t.pdate))

2024-01-07 <class 'datetime.date'>
2024-01-21 <class 'datetime.date'>
2024-02-07 <class 'datetime.date'>
2024-02-21 <class 'datetime.date'>


In [71]:
for t in Transaction.objects(description='Safeway'):
    print(t.originals)

weekly/Safeway
big party this weekend/Safeway
weekly/Safeway
normal/Safeway


In [72]:
lst = list(Transaction.objects(description='Safeway'))

In [73]:
lst[1].comment

''

In [74]:
lst[1].pamount

175.0

In [75]:
lst[1].pdate

datetime.date(2024, 1, 21)

In [76]:
list(Transaction.objects(pamount__lt=175.0))

[<Transaction: <Tr 2024-01-10 chase:visa -> car $50.0 Shell Oil>>,
 <Transaction: <Tr 2024-02-26 chase:visa -> car $60.0 Shell Oil>>,
 <Transaction: <Tr 2024-01-07 bank:checking -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-02-07 bank:checking -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-02-21 chase:visa -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-03-10 amex:blue -> household $50.0 Home Depot>>,
 <Transaction: <Tr 2024-03-12 household -> amex:blue $25.0 Home Depot>>]

In [77]:
for t in Transaction.objects(pdate=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

2024-01-02 5000.0 bank:checking yoyodyne
2024-01-02 5000.0 yoyodyne groceries/mortgage/car/travel


In [78]:
for t in Transaction.objects(pdate__lte=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

2024-01-02 5000.0 bank:checking yoyodyne
2024-01-02 5000.0 yoyodyne groceries/mortgage/car/travel


### Operators

In [79]:
for t in Transaction.objects(description__gte='Safeway'):
    print(t.pdate, t.description)

2024-01-02 paycheck
2024-01-02 fill buckets
2024-02-02 paycheck
2024-02-02 fill buckets
2024-01-05 car payment
2024-02-05 car payment
2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


In [80]:
for t in Transaction.objects(description__regex='^S'):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


The operator automatically applies to list elements.

In [81]:
for t in Transaction.objects(description__regex=r'\s'):
    print(t.pdate, t.description, t.pamount)

2024-01-02 fill buckets 5000.0
2024-02-02 fill buckets 5000.0
2024-01-05 car payment 500.0
2024-02-05 car payment 500.0
2024-01-10 Shell Oil 50.0
2024-02-26 Shell Oil 60.0
2024-01-04 Rocket Mortgage 1800.0
2024-02-04 Rocket Mortgage 1800.0
2024-03-10 Home Depot 50.0
2024-03-12 Home Depot 25.0


For compound constraints we need another class from MongoEngine.

In [82]:
from mongoengine.queryset.visitor import Q

In [83]:
for t in Transaction.objects(Q(description__regex=r'^S')):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


In [84]:
for t in Transaction.objects(Q(description__regex=r'^S') & Q(description__regex=r'\s')):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil


### QuerySet

In [85]:
for a in Account.nominal_accounts:
    print(a.name)

### Combining Query Elements

In [86]:
q = Q(description__regex=r'^S')

In [87]:
q

Q(**{'description__regex': '^S'})

In [88]:
type(q)

mongoengine.queryset.visitor.Q

In [89]:
p = Q(description__regex=r'\s')

In [90]:
p & q

(Q(**{'description__regex': '\\s'}) & Q(**{'description__regex': '^S'}))

In [91]:
for t in Transaction.objects(p & q):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil


Create Q object using dictionaries

In [92]:
dct = {'description__regex': r'^S'}

In [93]:
Q(**dct)

Q(**{'description__regex': '^S'})

Can an object have multiple constraints?

In [94]:
dct = {'description__regex': r'^S', 'pamount__gt': 100}

In [95]:
Q(**dct)

Q(**{'description__regex': '^S', 'pamount__gt': 100})

In [96]:
for t in Transaction.objects(Q(**dct)):
    print(t.pdate, t.description, t.pamount)

2024-01-21 Safeway 175.0


Yep!

### Select Method

#### Select Transactions

All transactions:

In [97]:
for t in DB.select(Transaction):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne bank:checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 yoyodyne bank:checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 bank:checking car
2024-02-05 500.0 bank:checking car
2024-01-10 50.0 chase:visa car
2024-02-26 60.0 chase:visa car
2024-01-04 1800.0 bank:checking mortgage
2024-02-04 1800.0 bank:checking mortgage
2024-01-07 75.0 bank:checking groceries
2024-01-21 175.0 chase:visa groceries
2024-02-07 75.0 bank:checking groceries
2024-02-21 75.0 chase:visa groceries
2024-03-10 50.0 amex:blue household
2024-03-12 25.0 household amex:blue


By date:

In [98]:
for t in DB.select(Transaction, date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-21 175.0 chase:visa groceries


In [99]:
for t in DB.select(Transaction, start_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-02-02 5000.0 yoyodyne bank:checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-05 500.0 bank:checking car
2024-02-26 60.0 chase:visa car
2024-02-04 1800.0 bank:checking mortgage
2024-01-21 175.0 chase:visa groceries
2024-02-07 75.0 bank:checking groceries
2024-02-21 75.0 chase:visa groceries
2024-03-10 50.0 amex:blue household
2024-03-12 25.0 household amex:blue


In [100]:
for t in DB.select(Transaction, end_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne bank:checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 bank:checking car
2024-01-10 50.0 chase:visa car
2024-01-04 1800.0 bank:checking mortgage
2024-01-07 75.0 bank:checking groceries
2024-01-21 175.0 chase:visa groceries


By amount:

In [101]:
for t in DB.select(Transaction, amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-07 75.0 bank:checking groceries
2024-02-07 75.0 bank:checking groceries
2024-02-21 75.0 chase:visa groceries


In [102]:
lst = DB.select(Transaction, amount=75)

In [103]:
all(t.pamount == 75 for t in lst)

True

In [104]:
for t in DB.select(Transaction, max_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-10 50.0 chase:visa car
2024-02-26 60.0 chase:visa car
2024-01-07 75.0 bank:checking groceries
2024-02-07 75.0 bank:checking groceries
2024-02-21 75.0 chase:visa groceries
2024-03-10 50.0 amex:blue household
2024-03-12 25.0 household amex:blue


In [105]:
for t in DB.select(Transaction, min_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne bank:checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 yoyodyne bank:checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 bank:checking car
2024-02-05 500.0 bank:checking car
2024-01-04 1800.0 bank:checking mortgage
2024-02-04 1800.0 bank:checking mortgage
2024-01-07 75.0 bank:checking groceries
2024-01-21 175.0 chase:visa groceries
2024-02-07 75.0 bank:checking groceries
2024-02-21 75.0 chase:visa groceries


By description:

In [106]:
for t in DB.select(Transaction, description = r'^s'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-01-10 50.0 Shell Oil chase:visa car
2024-02-26 60.0 Shell Oil chase:visa car
2024-01-07 75.0 Safeway bank:checking groceries
2024-01-21 175.0 Safeway chase:visa groceries
2024-02-07 75.0 Safeway bank:checking groceries
2024-02-21 75.0 Safeway chase:visa groceries


In [107]:
for t in DB.select(Transaction, comment=r'budget'):
    print(t.pdate, t.pamount, t.description, t.comment, t.pcredit, t.pdebit)

2024-01-02 5000.0 fill buckets 2024 budget groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 fill buckets 2024 budget groceries/mortgage/car/travel yoyodyne


By account:

In [108]:
for t in DB.select(Transaction, debit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-04 1800.0 bank:checking mortgage
2024-02-04 1800.0 bank:checking mortgage


In [109]:
for t in DB.select(Transaction, credit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne


Some random combinations

In [110]:
for t in DB.select(Transaction, description = r'^s', min_amount=100):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-01-21 175.0 Safeway chase:visa groceries


In [111]:
for t in DB.select(Transaction, start_date = date(2024,2,1), credit='visa'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-02-26 60.0 Shell Oil chase:visa car
2024-02-21 75.0 Safeway chase:visa groceries


#### Select Entries

All entries:

In [112]:
len(DB.select(Entry))

38

In [113]:
for e in DB.select(Entry):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 bank:checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 bank:checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 bank:checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 bank:checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 chase:visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 chase:visa 60.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 bank:checking 1800.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 bank:checking 1800.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 bank:checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 chase:visa 175.0 credit
202

By date:

In [114]:
for e in DB.select(Entry, date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-01-05 car 500.0 debit
2024-01-05 bank:checking 500.0 credit


In [115]:
for e in DB.select(Entry, start_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-02-02 bank:checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 bank:checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 bank:checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 chase:visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 chase:visa 60.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 bank:checking 1800.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 bank:checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 chase:visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 bank:checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 chase:visa 75.0 credit
2024-03-10 household 50.0 debit
2024-03-10 amex:blue 50.0 credit
2024-03-12 household 25.0 credit
2024-03-12 amex:blue 25.0 debit


In [116]:
for e in DB.select(Entry, end_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 bank:checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 bank:checking 500.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 bank:checking 1800.0 credit


By amount:

In [117]:
for e in DB.select(Entry, amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 car 900.0 credit
2024-02-02 car 900.0 credit


In [118]:
for e in DB.select(Entry, max_amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 groceries 300.0 credit
2024-01-02 car 900.0 credit
2024-02-02 groceries 300.0 credit
2024-02-02 car 900.0 credit
2024-01-05 car 500.0 debit
2024-01-05 bank:checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 bank:checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 chase:visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 chase:visa 60.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 bank:checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 chase:visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 bank:checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 chase:visa 75.0 credit
2024-03-10 household 50.0 debit
2024-03-10 amex:blue 50.0 credit
2024-03-12 household 25.0 credit
2024-03-12 amex:blue 25.0 debit


In [119]:
for e in DB.select(Entry, min_amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 bank:checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 bank:checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 bank:checking 1800.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 bank:checking 1800.0 credit


By account:

In [120]:
for e in DB.select(Entry, account='medical'):
    print(e.date, e.account, e.amount, e.column)

By column:

In [121]:
for e in DB.select(Entry, column='credit'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 yoyodyne 5000.0 credit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 bank:checking 500.0 credit
2024-02-05 bank:checking 500.0 credit
2024-01-10 chase:visa 50.0 credit
2024-02-26 chase:visa 60.0 credit
2024-01-04 bank:checking 1800.0 credit
2024-02-04 bank:checking 1800.0 credit
2024-01-07 bank:checking 75.0 credit
2024-01-21 chase:visa 175.0 credit
2024-02-07 bank:checking 75.0 credit
2024-02-21 chase:visa 75.0 credit
2024-03-10 amex:blue 50.0 credit
2024-03-12 household 25.0 credit


In [122]:
for e in DB.select(Entry, column='debit'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 bank:checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 debit
2024-02-02 bank:checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 debit
2024-01-05 car 500.0 debit
2024-02-05 car 500.0 debit
2024-01-10 car 50.0 debit
2024-02-26 car 60.0 debit
2024-01-04 mortgage 1800.0 debit
2024-02-04 mortgage 1800.0 debit
2024-01-07 groceries 75.0 debit
2024-01-21 groceries 175.0 debit
2024-02-07 groceries 75.0 debit
2024-02-21 groceries 75.0 debit
2024-03-10 household 50.0 debit
2024-03-12 amex:blue 25.0 debit


By tag:

In [123]:
for e in DB.select(Entry, tag='unpaired'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 bank:checking 5000.0 debit


### Serializing Objects

In [124]:
import json
from bson.objectid import ObjectId
import datetime

In [125]:
lst = DB.select(Transaction, start_date = date(2024,2,1), credit='visa')

In [126]:
lst[0].to_json()

'{"_id": {"$oid": "67f1617abcd0bf7da28fa486"}, "description": "Shell Oil", "comment": "", "tags": [], "entries": [{"$oid": "67f1617abcd0bf7da28fa46d"}, {"$oid": "67f1617abcd0bf7da28fa46e"}], "pdate": {"$date": 1708905600000}, "pdebit": "car", "pcredit": "chase:visa", "pamount": 60.0}'

In [127]:
type(lst[0])

dexter.DB.Transaction

In [128]:
obj = Transaction.objects.as_pymongo()[0]

In [129]:
type(obj)

dict

In [130]:
obj

{'_id': ObjectId('67f1617abcd0bf7da28fa47f'),
 'description': 'paycheck',
 'comment': '',
 'tags': [],
 'entries': [ObjectId('67f16179bcd0bf7da28fa459'),
  ObjectId('67f1617abcd0bf7da28fa45a')],
 'pdate': datetime.datetime(2024, 1, 2, 0, 0),
 'pdebit': 'bank:checking',
 'pcredit': 'yoyodyne',
 'pamount': 5000.0}

In [131]:
s = lst[0].to_json()

In [132]:
json.loads(s)

{'_id': {'$oid': '67f1617abcd0bf7da28fa486'},
 'description': 'Shell Oil',
 'comment': '',
 'tags': [],
 'entries': [{'$oid': '67f1617abcd0bf7da28fa46d'},
  {'$oid': '67f1617abcd0bf7da28fa46e'}],
 'pdate': {'$date': 1708905600000},
 'pdebit': 'car',
 'pcredit': 'chase:visa',
 'pamount': 60.0}

In [133]:
Transaction.from_json(s)

<Transaction: <Tr 2024-02-26 chase:visa -> car $60.0 Shell Oil>>

In [134]:
s = 'account: {...:...}'

In [135]:
s.find(':')

7

In [136]:
s[:s.find(':')]

'account'

In [137]:
s[s.find(':'):]

': {...:...}'

### Indexes

We want a field in Entry documents that serves as a unique ID so we can tell if an item was imported before.

MongoEngine has a UUID field.
* how is it computed?  is it a hash of all the other field values?
* when is it computed?  when the object is made, or when it is saved?

In [138]:
from mongoengine import *

In [139]:
class Foo(Document):
    name = StringField()
    amount = FloatField()
    uid = UUIDField(binary=False)

In [140]:
f = Foo(name='Fred', amount=10)

Just declaring it is not enough to give it a value:

In [141]:
f.uid is None

True

In [142]:
f.save()

<Foo: Foo object>

This model has an index.  The `#` means it's a "hashed index" but no discussion of what that means or why we'd want one (over say a text index that we compute ourselves).

In [143]:
class Bar(Document):
    name = StringField()
    amount = FloatField()
    uid = UUIDField(binary=False)
    meta = {
        'indexes': ['#uid']
    }

In [144]:
b1 = Bar(name='george', amount=20, uid='123')

In [145]:
b1.uid

'123'

Ah -- the UUID is created when the document is saved (the same was true for Foo, above, if a `uid` value passed to the constructor).

```
b1.save()
...
ValidationError: ValidationError (Bar:None) (Could not convert to UUID: badly formed hexadecimal UUID string: ['uid'])
```

So how do we make a UUID?  Do we care?  Why not just use our hashed strings?

#### UUID

It's in the PyMongo docs (and we had to specify how they're represented when we made the DB connection).  It's also a Python library.

In [146]:
from uuid import uuid4

In [147]:
b2 = Bar(name='ringo', amount=30, uid=uuid4())

In [148]:
b2.uid

UUID('3f7011dc-4ca5-4444-ab31-4eda72db1667')

In [149]:
b2.save()

<Bar: Bar object>

In [150]:
x = b2.uid

In [151]:
b3 = Bar(name='paul', amount=40, uid=x)

In [152]:
b3.save()

<Bar: Bar object>

In [153]:
b3.uid == b2.uid

True

In [154]:
uuid4()

UUID('93b6c212-6d0f-4d5d-a8b2-81f04a613b2f')

So just defining an index isn't enough to make it unique.

### Unique

In [155]:
class FooBar(Document):
    name = StringField()
    amount = FloatField()
    uid = StringField(unique=True)

In [156]:
f1 = FooBar(name='Fred', amount=100, uid='bedrock')

In [157]:
f2 = FooBar(name='Barney', amount=200, uid='bedrock')

If uncommented this cell will add a record to the database.  To reset the database to its original condition run pytest again.

In [158]:
# f1.save()

In [159]:
# try:
#     f2.save()
# except NotUniqueError as err:
#     print(err)

In [160]:
FooBar._meta

{'abstract': False,
 'max_documents': None,
 'max_size': None,
 'ordering': [],
 'indexes': [],
 'id_field': 'id',
 'index_background': False,
 'index_opts': None,
 'delete_rules': None,
 'allow_inheritance': None,
 'collection': 'foo_bar',
 'index_specs': [{'fields': [('uid', 1)], 'unique': True, 'sparse': False}]}

Awesome!  Just defining a field as unique is enough to have MongoEngine create an index.  Don't know (and don't care, yet, at least) about the ramifications of `sparse = False`.

> checked PyMongo docs, it's not what we think, and not something we want (even though we can have it by specifying `sparse=True` in the column spec)

### Match Account Names

In [161]:
Account.objects(name__contains='checking')

[<Account: <Acct bank:checking asset>>]

In [162]:
Account.objects(name__contains='g')

[<Account: <Acct bank:checking asset>>, <Account: <Acct groceries expense>>, <Account: <Acct mortgage expense>>]

In [163]:
DB.find_account('g')

[<Account: <Acct bank:checking asset>>, <Account: <Acct groceries expense>>, <Account: <Acct mortgage expense>>]

In [164]:
DB.account_name_parts()

{'amex',
 'bank',
 'blue',
 'car',
 'chase',
 'checking',
 'equity',
 'groceries',
 'household',
 'mortgage',
 'travel',
 'visa',
 'yoyodyne'}

In [165]:
DB.account_name_parts('expense')

{'car', 'groceries', 'household', 'mortgage', 'travel'}

In [166]:
DB.full_names()

{'equity': ['equity', 'equity', 'equity'],
 'yoyodyne': ['yoyodyne'],
 'bank': ['bank:checking'],
 'checking': ['bank:checking'],
 'amex': ['amex:blue'],
 'blue': ['amex:blue'],
 'chase': ['chase:visa'],
 'visa': ['chase:visa'],
 'groceries': ['groceries'],
 'household': ['household'],
 'mortgage': ['mortgage'],
 'car': ['car'],
 'travel': ['travel']}

In [167]:
DB.full_names('expense')

{'groceries': ['groceries'],
 'household': ['household'],
 'mortgage': ['mortgage'],
 'car': ['car'],
 'travel': ['travel']}

### Aggregation

In [169]:
Entry.objects.sum('amount')

50370.0

In [171]:
Entry.objects(column='credit')

[<Entry: <En 2024-01-02 yoyodyne -$5000.0>>, <Entry: <En 2024-01-02 groceries -$300.0>>, <Entry: <En 2024-01-02 mortgage -$1800.0>>, <Entry: <En 2024-01-02 car -$900.0>>, <Entry: <En 2024-01-02 travel -$2000.0>>, <Entry: <En 2024-02-02 yoyodyne -$5000.0>>, <Entry: <En 2024-02-02 groceries -$300.0>>, <Entry: <En 2024-02-02 mortgage -$1800.0>>, <Entry: <En 2024-02-02 car -$900.0>>, <Entry: <En 2024-02-02 travel -$2000.0>>, <Entry: <En 2024-01-05 bank:checking -$500.0>>, <Entry: <En 2024-02-05 bank:checking -$500.0>>, <Entry: <En 2024-01-10 chase:visa -$50.0>>, <Entry: <En 2024-02-26 chase:visa -$60.0>>, <Entry: <En 2024-01-04 bank:checking -$1800.0>>, <Entry: <En 2024-02-04 bank:checking -$1800.0>>, <Entry: <En 2024-01-07 bank:checking -$75.0>>, <Entry: <En 2024-01-21 chase:visa -$175.0>>, <Entry: <En 2024-02-07 bank:checking -$75.0>>, <Entry: <En 2024-02-21 chase:visa -$75.0>>, '...(remaining elements truncated)...']

In [172]:
Entry.objects(column='credit').sum('amount')

25185.0

In [176]:
Entry.objects(Q(account="groceries") & Q(column='credit')).sum('amount')

600.0

In [174]:
for e in Entry.objects(column='credit'):
    print(e)

<En 2024-01-02 yoyodyne -$5000.0>
<En 2024-01-02 groceries -$300.0>
<En 2024-01-02 mortgage -$1800.0>
<En 2024-01-02 car -$900.0>
<En 2024-01-02 travel -$2000.0>
<En 2024-02-02 yoyodyne -$5000.0>
<En 2024-02-02 groceries -$300.0>
<En 2024-02-02 mortgage -$1800.0>
<En 2024-02-02 car -$900.0>
<En 2024-02-02 travel -$2000.0>
<En 2024-01-05 bank:checking -$500.0>
<En 2024-02-05 bank:checking -$500.0>
<En 2024-01-10 chase:visa -$50.0>
<En 2024-02-26 chase:visa -$60.0>
<En 2024-01-04 bank:checking -$1800.0>
<En 2024-02-04 bank:checking -$1800.0>
<En 2024-01-07 bank:checking -$75.0>
<En 2024-01-21 chase:visa -$175.0>
<En 2024-02-07 bank:checking -$75.0>
<En 2024-02-21 chase:visa -$75.0>
<En 2024-03-10 amex:blue -$50.0>
<En 2024-03-12 household -$25.0>
